In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_formatting import *
from collections import Counter
import numpy as np
import time

import pandas as pd
import datetime as dt
import requests
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
from matplotlib import ticker
from matplotlib.ticker import MaxNLocator

from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import itertools as it

plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor']='white'
plt.rcParams['savefig.facecolor']='white'
plt.rc('axes', axisbelow=True)
%matplotlib inline

In [3]:
today = time.strftime("%Y-%m-%d")

month_dict = {'01':'Jan','02':'Feb','03':'Mar','04':'Apr','05':'May','06':'Jun',
              '07':'Jul','08':'Aug','09':'Sep','10':'Oct','11':'Nov','12':'Dec'}

weekday_mapping = {0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}

alldates = pd.date_range(start='2020-01-06', end='2021-12-31').tolist()
datelist = [str(i.date()) for i in alldates]
weekdays = [weekday_mapping[i.date().weekday()] for i in alldates]
weeklist = [i//7 for i in range(len(alldates))]
DATE_DF = pd.DataFrame({'date':datelist,'day':weekdays,'week':weeklist})
DATERANGE = DATE_DF['date'].tolist()

date_dict = {DATERANGE[i]:i for i in range(len(DATERANGE))}

In [4]:
usr_agent = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/61.0.3163.100 Safari/537.36'}

# Alabama
https://services7.arcgis.com/4RQmZZ0yaZkGR1zy/arcgis/rest/services/alsde_c19_publish_PUBLIC/FeatureServer/0/query?f=json&where=WeekOf%3D9&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=*&orderByFields=SchoolSystem%20asc&resultOffset=0&resultRecordCount=200&resultType=standard&cacheHint=true

https://services7.arcgis.com/4RQmZZ0yaZkGR1zy/arcgis/rest/services/alsde_c19_publish_PUBLIC/FeatureServer/0/query?f=json&where=WeekOf%3D9&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=*&orderByFields=SchoolSystem%20asc&outSR=102100&resultOffset=0&resultRecordCount=150&resultType=standard&cacheHint=true

# Alaska


# Arizona

https://www.azed.gov/covid-19/learning-model-tracker?page=1

In [5]:
out_az = []
for page_i in range(19):
    az_school_url_i = 'https://www.azed.gov/covid-19/learning-model-tracker?page=%i'%page_i
    page = requests.get(az_school_url_i)
    soup = BeautifulSoup(page.content, 'html.parser')
    tbl = soup.find("table",{"class":"views-table cols-3 table table-hover table-striped"})
    df_tbl = pd.read_html(str(tbl))[0]
    out_az.append(df_tbl)
    
az_df_status = pd.concat(out_az).reset_index().iloc[:,1:]
az_df_status

,LEA Name,County,Status
0,La Paloma Academy Central,Pima,Distance
1,A+ Charter Schools,Pinal,Hybrid
2,Academy Del Sol,Pima,Hybrid
3,Academy Del Sol - Hope,Pima,Hybrid
4,Academy of Building Industries,Mohave,Hybrid
...,...,...,...
449,Young Scholars Academy,Mohave,In-Person
450,Yucca Elementary School,Mohave,Hybrid
451,Yuma Private Industry Council,Yuma,Distance
452,Yuma School District One,Yuma,Distance


# Arkansas

schools: https://www.healthy.arkansas.gov/images/uploads/pdf/School_Report_02252021.pdf

schools: https://www.healthy.arkansas.gov/images/uploads/pdf/School_Report_03012021.pdf

outbreaks: https://www.healthy.arkansas.gov/images/uploads/pdf/Active_COVID-19_report02252021.pdf

https://achi.net/covid19/

In [5]:
import tabula

In [6]:
ar_dates = DATE_DF.loc[(DATE_DF['date']<=today)&(DATE_DF['date']>'2020-08-01')]['date'].values
web_url = 'http://web.archive.org'

In [7]:
ar_out = []
checks = []
# for i in range(30):
for i in range(len(ar_dates)):
    try:
        di = ar_dates[i].replace('-','')
        ar_school_url_i = web_url+'/web/%s/https://www.healthy.arkansas.gov/programs-services/topics/covid-19-reports'%di
        page = requests.get(ar_school_url_i)
        soup = BeautifulSoup(page.content, 'html.parser')

        links_i = [i.get('href') for i in soup.find_all('a')]
        links_i = [i for i in links_i if 'school' in str(i).lower()]
        for x in links_i:
            add_url = web_url+x
            if add_url.split('/pdf/')[-1] not in checks:
                print(ar_dates[i],add_url.split('/pdf/')[-1])
                checks.append(add_url.split('/pdf/')[-1])
                ar_out.append(add_url)
    except:
        continue

di = today.replace('-','')
ar_school_url_i = 'https://www.healthy.arkansas.gov/programs-services/topics/covid-19-reports'

page = requests.get(ar_school_url_i)
soup = BeautifulSoup(page.content, 'html.parser')

links_i = [i.get('href') for i in soup.find_all('a')]
links_i = [i for i in links_i if 'school' in str(i).lower()]
for x in links_i:
    add_url = web_url+x
    if add_url.split('/pdf/')[-1] not in checks:
        print(today,add_url)
        checks.append(add_url.split('/pdf/')[-1])
        ar_out.append(add_url)

2020-08-27 school_report_8_27.pdf
2020-09-01 school_report_08312020.pdf
2020-09-05 School_Report_09072020.pdf
2020-09-05 School_Report_09032020.pdf
2020-09-15 School_Report_09172020.pdf
2020-09-15 School_Report_09142020_(002).pdf
2020-09-22 School_Report_09212020.pdf
2020-09-26 School_Report_09242020_(002).pdf
2020-09-30 School_Report_10012020.pdf
2020-09-30 School_Report_09282020.pdf
2020-10-07 School_Report_10082020.pdf
2020-10-07 School_Report_10052020.pdf
2020-10-14 School_Report_10152020.pdf
2020-10-14 School_Report_10122020.pdf
2020-10-19 School_Report_10192020.pdf
2020-10-24 School_Report_10262020.pdf
2020-10-24 School_Report_10222020.pdf
2020-10-29 School_Report_10292020.pdf
2020-11-03 School_Report_11022020.pdf
2020-11-06 School_Report_11052020.pdf
2020-11-10 School_Report_11092020.pdf
2020-11-14 School_Report_11162020.pdf
2020-11-14 School_Report_11112020.pdf
2020-11-19 School_Report_11192020.pdf
2020-11-26 School_Report_11252020.pdf
2020-11-26 School_Report_11232020.pdf
2020

In [8]:
ar_out

['http://web.archive.org/web/20200828141222/https://www.healthy.arkansas.gov/images/uploads/pdf/school_report_8_27.pdf',
 'http://web.archive.org/web/20200901231530/https://www.healthy.arkansas.gov/images/uploads/pdf/school_report_08312020.pdf',
 'http://web.archive.org/web/20200909055717/https://www.healthy.arkansas.gov/images/uploads/pdf/School_Report_09072020.pdf',
 'http://web.archive.org/web/20200909055717/https://www.healthy.arkansas.gov/images/uploads/pdf/School_Report_09032020.pdf',
 'http://web.archive.org/web/20200921073908/https://www.healthy.arkansas.gov/images/uploads/pdf/School_Report_09172020.pdf',
 'http://web.archive.org/web/20200921073908/https://www.healthy.arkansas.gov/images/uploads/pdf/School_Report_09142020_(002).pdf',
 'http://web.archive.org/web/20200924115948/https://www.healthy.arkansas.gov/images/uploads/pdf/School_Report_09212020.pdf',
 'http://web.archive.org/web/20200928144404/https://www.healthy.arkansas.gov/images/uploads/pdf/School_Report_09242020_(002

In [26]:
ar_file_i = web_url+links_i[1]
tabula.read_pdf(ar_file_i,pages=1)[0]

Error from tabula-java:
Error: Error: End-of-File, expected line




CalledProcessError: Command '['java', '-Djava.awt.headless=true', '-Dfile.encoding=UTF8', '-jar', '/Users/brennan/anaconda3/envs/covid/lib/python3.8/site-packages/tabula/tabula-1.0.4-jar-with-dependencies.jar', '--pages', '1', '--guess', '--format', 'JSON', 'school_report_8_27.pdf']' returned non-zero exit status 1.

In [ ]:
# ar_tmp = []
# for i in range(len(ar_dates)):
#     di = ar_dates[i][5:7]+ar_dates[i][-2:]+ar_dates[i][:4]
#     file_i = 'https://www.healthy.arkansas.gov/images/uploads/pdf/School_Report_%s.pdf'%di
#     try:
#         ar_df_i = tabula.read_pdf(file_i,pages=1,multiple_tables=False)[0]
#         ar_tmp.append(file_i)
#         print('yes',ar_dates[i])
#     except:
#         di1 = ar_dates[i][5:7]+ar_dates[i][-2:]+ar_dates[i][:4]
#         di2 = ar_dates[i].replace('-','')
#         fi2 = 'http://web.archive.org/web/%s/https://www.healthy.arkansas.gov/images/uploads/pdf/School_Report_%s.pdf'%(di2,di1)
#         print('...trying',fi2)
#         try:
#             ar_df_i = tabula.read_pdf(fi2,pages=1,multiple_tables=False)[0]
#             ar_tmp.append(fi2)
#             print('yes',ar_dates[i])
#         except:
#             continue
#             print('\tno',ar_dates[i])

# California

https://maps.schools.covid19.ca.gov/public.html

# Colorado

https://docs.google.com/spreadsheets/d/e/2PACX-1vQipdjO8QWhilhhJ4bX0FBebnHEzK1G3LEDQbE_S-xRvs2t0oHNm--acHwMRFmL9uKw4cXcOUqy1V66/pubhtml#

# Connecticut 

https://data.ct.gov/Health-and-Human-Services/COVID-19-Cases-in-CT-Schools-By-School-/u8jq-fxc2

# Delaware 

https://myhealthycommunity.dhss.delaware.gov/locations/state/in-person-contagious/district-school-cases/range/2021-01-16/2021-01-22
https://myhealthycommunity.dhss.delaware.gov/locations/state/in-person-contagious/setting-school-cases

https://myhealthycommunity.dhss.delaware.gov/locations/state/covid_venue_reports/2021-01-24/2021-01-30

<option value='2021-02-20 - 2021-02-26'>02/20/2021 - 02/26/2021</option>
<option value='2021-02-13 - 2021-02-19'>02/13/2021 - 02/19/2021</option>
<option value='2021-02-06 - 2021-02-12'>02/06/2021 - 02/12/2021</option>
<option value='2021-01-30 - 2021-02-05'>01/30/2021 - 02/05/2021</option>
<option value='2021-01-23 - 2021-01-29'>01/23/2021 - 01/29/2021</option>
<option value='2021-01-16 - 2021-01-22'>01/16/2021 - 01/22/2021</option>
<option value='2021-01-09 - 2021-01-15'>01/09/2021 - 01/15/2021</option>
<option value='2021-01-02 - 2021-01-08'>01/02/2021 - 01/08/2021</option>
<option value='2020-12-26 - 2021-01-01'>12/26/2020 - 01/01/2021</option>
<option value='2020-12-19 - 2020-12-25'>12/19/2020 - 12/25/2020</option>
<option value='2020-12-12 - 2020-12-18'>12/12/2020 - 12/18/2020</option>
<option value='2020-12-05 - 2020-12-11'>12/05/2020 - 12/11/2020</option>
<option value='2020-11-28 - 2020-12-04'>11/28/2020 - 12/04/2020</option>
<option value='2020-11-21 - 2020-11-27'>11/21/2020 - 11/27/2020</option>
<option value='2020-11-14 - 2020-11-20'>11/14/2020 - 11/20/2020</option>
<option value='2020-11-07 - 2020-11-13'>11/07/2020 - 11/13/2020</option>
<option value='2020-10-31 - 2020-11-06'>10/31/2020 - 11/06/2020</option>
<option value='2020-10-24 - 2020-10-30'>10/24/2020 - 10/30/2020</option>
<option value='2020-10-17 - 2020-10-23'>10/17/2020 - 10/23/2020</option>
<option value='2020-10-10 - 2020-10-16'>10/10/2020 - 10/16/2020</option>
<option value='2020-10-03 - 2020-10-09'>10/03/2020 - 10/09/2020</option>
<option value='2020-09-26 - 2020-10-02'>09/26/2020 - 10/02/2020</option>
<option value='2020-09-19 - 2020-09-25'>09/19/2020 - 09/25/2020</option>
<option value='2020-09-12 - 2020-09-18'>09/12/2020 - 09/18/2020</option>
<option value='2020-09-05 - 2020-09-11'>09/05/2020 - 09/11/2020</option>
<option value='2020-08-29 - 2020-09-04'>08/29/2020 - 09/04/2020</option>

# District of Columbia

https://coronavirus.dc.gov/page/outbreak-data

# Florida
http://ww11.doh.state.fl.us/comm/_partners/covid19_report_archive/school-reports/schools_latest.pdf


# Georgia

https://dph.georgia.gov/school-aged-covid-19-surveillance-data

# Hawaii



# Idaho

https://coronavirus.idaho.gov/resources-for-schools/

https://coronavirus.idaho.gov/wp-content/uploads/2021/03/School-Report-3.1.21-Fixed.pdf

https://services7.arcgis.com/ugRfT6qLoOKPTf05/arcgis/rest/services/Idaho_School_Districts_View_Layer/FeatureServer/0/query?f=json&where=status%20%3D%20%27IN%20PERSON%27&returnGeometry=true&spatialRel=esriSpatialRelIntersects&maxAllowableOffset=2445&geometry=%7B%22xmin%22%3A-13775786.985668924%2C%22ymin%22%3A5009377.085700948%2C%22xmax%22%3A-12523442.714244926%2C%22ymax%22%3A6261721.357124943%2C%22spatialReference%22%3A%7B%22wkid%22%3A102100%7D%7D&geometryType=esriGeometryEnvelope&inSR=102100&outFields=*&outSR=102100&resultType=tile


https://services7.arcgis.com/ugRfT6qLoOKPTf05/arcgis/rest/services/Idaho_School_Districts_View_Layer/FeatureServer/0/query?f=json&where=status%20%3D%20%27HYBRID%27&returnGeometry=true&spatialRel=esriSpatialRelIntersects&maxAllowableOffset=2445&geometry=%7B%22xmin%22%3A-13775786.985668924%2C%22ymin%22%3A5009377.085700948%2C%22xmax%22%3A-12523442.714244926%2C%22ymax%22%3A6261721.357124943%2C%22spatialReference%22%3A%7B%22wkid%22%3A102100%7D%7D&geometryType=esriGeometryEnvelope&inSR=102100&outFields=*&outSR=102100&resultType=tile



# Illinois

http://web.archive.org/web/*/https://www.dph.illinois.gov/covid19/location-exposure?regionID=0

https://idph.illinois.gov/DPHPublicInformation/api/covidct/GetCOVIDSchoolExposures

https://idph.illinois.gov/DPHPublicInformation/api/COVID/GetSchoolOutbreaks

https://isp.maps.arcgis.com/apps/opsdashboard/index.html#/fceeacb37da04de4b237ed941dd7d5c4

# Indiana
Not available at this time		Indiana COVID-19 Data Report	Use "School" tab to see the data that is updated every Monday and is broken down by student, teacher, and staff.

https://www.coronavirus.in.gov/map/covid-19-indiana-school-report.json




# Iowa
Summary of Schools Providing Primarily Continuous/Remote Learning		Not available at this time	The state has a dashboard that shows the past 14 day average positivity rates by county that school districts can use.

https://educateiowa.gov/covid-19/requests-move-remote-learning-school-building-or-district


# Kansas
Not available at this time		Not available at this time

# Kentucky
Not available at this time		K-12 School COVID-19 Self-Reported Data

https://public.tableau.com/profile/chfs.dph#!/vizhome/COVID19SchoolSelfReportngData/SchoolSelfReportCovid19DB

# Louisiana
Learning Environment and Connectivity Tool		COVID-19 Outbreaks - K-12 Schools Reporting	The Louisiana Department of Health's dashboard breaks cases down by parish.

https://ldh.la.gov/assets/oph/Coronavirus/resources/k-12/reporting/K-12ReportbyParish-02282021.pdf

https://datastudio.google.com/u/0/reporting/57592098-3276-452a-9872-456384c0acdc/page/2oidB?s=jF2UNzlqNKM

# Maine
Not available at this time		COVID-19 Cases in Maine Pre-K-12 Schools Over the Last 30 days	Updated on Thursdays at 4pm.


# Maryland
Positive COVID-19 Related Status of Instructional Models		MD COVID-19 SchoolOutbreak

https://coronavirus.maryland.gov/datasets/md-covid19-schooloutbreaks/data

http://www.marylandpublicschools.org/newsroom/_layouts/15/xlviewer.aspx?id=/newsroom/Documents/School-System-Instructional-Status-Form.xlsx

# Massachusetts
Not available at this time	The state has a list of district reopening models at the beginning of the academic year available here: 🔗	Positive COVID-19 Cases in Schools

https://www.doe.mass.edu/covid19/positive-cases/

# Michigan
Extended COVID-19 Learning Plan Dashboard		School-Related Outbreak Reporting	This page is updated every Monday.

https://www.mischooldata.org/covid-dashboard/

https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173_102480---,00.html

# Minnesota
Safe Learning Model Data	This page links to a dashboard that compiles instructional model data self-reported from districts and charter schools.	COVID-19 Situation Update	School buildings reporting 5 or more cases are listed in the "Facilities with Cases & Exposures" and more detailed data can be found in the weekly reports.

https://www.health.state.mn.us/diseases/coronavirus/situation.html#ccf1

https://education.mn.gov/MDE/dse/health/covid19/slmd/

# Mississippi
Not available at this time		COVID-19 Cases and Outbreaks in K-12 Schools

https://www.msdh.ms.gov/msdhsite/_static/14,0,420,972.html

# Missouri
Statewide Patterns of Instruction		COVID-19 in School District Communities

https://dese.mo.gov/communications/coronavirus-covid-19-information

https://showmestrong.mo.gov/data/public-health/school-districts/


# Montana
Not available at this time		Coronavirus Disease 2019 (COVID-19)	The COVID-19 in Montana Schools report, which is updated every Wednesday, can be accessed from the Coronavirus Disease 2019 (COVID-19) dashboard.




# Nebraska
Not available at this time		Not available at this time		


# Nevada
Not available at this time		Not available at this time		


# New Hampshire
COVID-19 Schools Dashboard		COVID-19 Schools Dashboard

https://www.nh.gov/covid19/dashboard/schools.htm#dash



# New Jersey
Not available at this time		COVID-19 Information Hub	Use the Schools tab to see the NJ COVID-19 School-Related Outbreaks Dashboard

https://covid19.nj.gov/forms/datadashboard


# New Mexico
School Reentry Status	Scroll down to the button for "School Reentry Status	Rapid Response Covid-19 Watchlist	For school-related cases, search for "NMPED" in the search bar.

https://www.env.nm.gov/rapid-response-data/

https://webnew.ped.state.nm.us/reentry-district-and-school-guidance/

# New York
Not available at this time		COVID-19 Report Card

https://schoolcovidreportcard.health.ny.gov/#/home


# North Carolina
NC School Reopenings for 2020-21		NC School Reopenings for 2020-21


https://docs.google.com/spreadsheets/d/1We8gDpa4Do5NR83Nf8niGE_YxzLDf-KZh-tVWifStxE/edit#gid=0


# North Dakota
Not available at this time		K-12 School Dashboard

https://www.health.nd.gov/k-12-school-dashboard


# Ohio
Education Model for School Year 2021	Scroll down to see the map and the link that shows current instructional models.	COVID-19 Dashboard

http://education.ohio.gov/Topics/Reset-and-Restart

https://coronavirus.ohio.gov/wps/portal/gov/covid-19/dashboards/schools-and-children/schools


# Oklahoma
Not available at this time		Not available at this time




# Oregon
School Status page	Updated on a weekly basis, this Department of Education data includes both private and public school information.	COVID-19 Reporting	Updated weekly, this page provides data on cases associated with schools that offer in-person instruction.

https://www.oregon.gov/ode/students-and-family/healthsafety/Pages/COVID-19-Reporting.aspx

https://www.oregon.gov/ode/students-and-family/healthsafety/Pages/2020-21-School-Status.aspx


# Pennsylvania
Not available at this time		Not available at this time




# Rhode Island

https://docs.google.com/spreadsheets/d/1c2QrNMz8pIbYEKzMJL7Uh2dtThOJa2j1sSMwiDo5Gz4/edit#gid=594871904

# South Carolina
School District COVID-19 Current Operational Status Map		COVID-19 Cases Associated with Staff & Students

http://web.archive.org/web/20210201230327/https://ed.sc.gov/districts-schools/schools/district-and-school-closures/operational-status/
https://scdhec.gov/covid19/resources-community-covid-19/covid-19-cases-associated-staff-students


# South Dakota
Not available at this time		South Dakota COVID-19 Updates	Scroll to the "South Dakota K-12 Schools and Colleges, Universities, and Technical Schools" report, which is updated on Mondays.

https://web.archive.org/web/20210128224326/https://doh.sd.gov/documents/COVID19/School-College_COVID_cases.pdf

# Tennessee
TDOE COVID-19 District Information	Districts are requested to submit COVID and instructional model data weekly.	TDOE COVID-19 District Information	Districts are requested to submit COVID and instructional model data weekly.	

https://districtinformation.tnedu.gov/api/districts

http://web.archive.org/web/20201105231752/https://districtinformation.tnedu.gov/api/districts

# Texas
Not available at this time	Texas requires school districts to provide daily, on-campus learning for any family who wants it in order to not lose state funding due to declines in enrollment.	Texas Public Schools COVID-19 Data	This report on public schools is updated on Fridays.

https://dshs.texas.gov/coronavirus/schools/texas-education-agency/


# Utah
Not available at this time		Case Counts	Use the schools tab to see the School-Associated Cases by School District report.	

https://coronavirus-dashboard.utah.gov/#schools

https://coronavirus-dashboard.utah.gov/

http://web.archive.org/web/2020*/https://coronavirus-dashboard.utah.gov/Utah_COVID19_data.zip

# Vermont
Not available at this time		PreK-12 Schools	Click on the "COVID-19 Cases in School While Infectious" link which is updated Tuesdays and Fridays.

https://www.healthvermont.gov/covid-19/your-community/prek-12-schools

http://web.archive.org/web/*/https://www.healthvermont.gov/sites/default/files/documents/pdf/COVID19-Transmission-Schools.pdf


# Virginia
State Snapshot: Virginia School Operational Status		COVID-19 Outbreaks by Selected Exposure Settings	This report is updated on Fridays.

https://www.doe.virginia.gov/support/health_medical/office/reopen-status.shtml

https://www.vdh.virginia.gov/coronavirus/covid-19-outbreaks-in-virginia-colleges-and-universities/




# Washington
School Reopening Data		Not available at this time

https://www.k12.wa.us/about-ospi/press-releases/novel-coronavirus-covid-19-guidance-resources/school-reopening-data

# West Virginia
Not available at this time		COVID-19 Outbreaks in State Public Schools

https://wvde.us/schooloutbreaks




# Wisconsin 
Not available at this time		COVID-19 Historical Data by School District

https://data.dhsgis.wi.gov/datasets/covid-19-historical-data-by-census-tract

https://data.dhsgis.wi.gov/datasets/covid-19-historical-data-by-school-district?orderBy=DATE&orderByAsc=false




# Wyoming
Not available at this time		Not available at this time		